## 2. Model Quantization

In [3]:
import os

# GPU 사용 설정
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import tensorflow as tf
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
print("TensorFlow version:", tf.__version__)

import numpy as np
from tensorflow import keras
from tensorflow_model_optimization.quantization.keras import vitis_quantize, vitis_inspect
import cv2


# paths
SCRIPT_DIR = os.getcwd()
FLOAT_DIR = os.path.join(SCRIPT_DIR, "float_model")
QUANT_DIR = os.path.join(SCRIPT_DIR, "quant_model")
LOG_DIR = os.path.join(SCRIPT_DIR, "logs")
DATASET_DIR = os.path.join(SCRIPT_DIR, "grocery_store_dataset")
TEST_DIR = os.path.join(DATASET_DIR, "new/test")
CALIB_DIR = os.path.join(DATASET_DIR, "new/calib")

# Ensure directories exist
os.makedirs(LOG_DIR, exist_ok=True)

# Load the trained model
FLOAT_HDF5_FILE = os.path.join(FLOAT_DIR, "train_resnet50_grocery_best_chkpt.h5")
model = keras.models.load_model(FLOAT_HDF5_FILE)

# label names
labelNames_list = ["Apple", "Juice", "Melon", "Milk", "Pear", "Pepper", "Tomato", "Yoghurt"]
labelNames_dict = {name: i for i, name in enumerate(labelNames_list)}

# quantized model path
QUANT_HDF5_FILE = os.path.join(QUANT_DIR, "quantized_resnet50_grocery.h5")

print("Setup complete.")

2024-07-30 22:08:45.954265: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-30 22:08:47.010030: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPU is NOT AVAILABLE
TensorFlow version: 2.12.0


2024-07-30 22:08:48.680486: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Setup complete.


In [4]:
# Load and preprocess data
def load_data(directory):
    img_list = [os.path.join(root, name)
                for root, _, files in os.walk(directory)
                for name in files
                if name.endswith((".jpg", ".jpeg", ".png"))]
    np.random.shuffle(img_list)
    x, y = list(), list()
    for img_path in img_list:
        classname = os.path.basename(os.path.dirname(img_path))
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))  # ResNet50 expects 224x224 images
        x.append(img)
        y.append(labelNames_dict[classname])
    return np.array(x), np.array(y)

x_calib, y_calib = load_data(CALIB_DIR)
x_test, y_test = load_data(TEST_DIR)

# Normalize images
x_calib = x_calib.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# One-hot encode labels
y_calib = keras.utils.to_categorical(y_calib, len(labelNames_list))
y_test = keras.utils.to_categorical(y_test, len(labelNames_list))

# Set batch size
BATCH_SIZE = 1 

# Create data generator for batch processing
def data_generator(x, y, batch_size):
    while True:
        for i in range(0, len(x), batch_size):
            yield x[i:i+batch_size], y[i:i+batch_size]

# Set the target for quantization
TARGET = "0x101000016010407"

# Perform quantization with target setting
quantizer = vitis_quantize.VitisQuantizer(model, target=TARGET)
q_model = quantizer.quantize_model(
    calib_dataset=x_calib,
    calib_batch_size=BATCH_SIZE,
    calib_steps=len(x_calib) // BATCH_SIZE
)

# Evaluate quantized model on test data
with vitis_quantize.quantize_scope():
    q_model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    q_eval_results = q_model.evaluate(
        data_generator(x_test, y_test, BATCH_SIZE),
        steps=len(x_test) // BATCH_SIZE
    )
    print("\n***************** Summary *****************")
    print(f"Test Quantized model accuracy: {q_eval_results[1]:.4f}")

# Save quantized model
q_model.save(QUANT_HDF5_FILE)
print(f"Quantized model saved to {QUANT_HDF5_FILE}")

# Perform model inspection on the quantized model
inspector = vitis_inspect.VitisInspector(target=TARGET)
filename_dump = os.path.join(LOG_DIR, "quantized_inspect_results.txt")
filename_svg = os.path.join(LOG_DIR, "quantized_model.svg")
inspector.inspect_model(
    float_model=q_model,
    input_shape=[1, 224, 224, 3],  # ResNet50 input size
    plot=True,
    plot_file=filename_svg,
    dump_results=True,
    dump_results_file=filename_dump,
    verbose=0
)
print(f"Quantized model inspection complete. Results saved to {filename_dump} and {filename_svg}")

[VAI INFO] Using func format quantizer
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 13s 1s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
16/16 [==============================] - 33s 382ms/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quant Model Refinement...
[VAI INFO] Start Quantize Position Ajustment...
[VAI INFO] Quantize Position Ajustment Done.
[VAI INFO] Post-Quant Model Refninement Done.
[VAI INFO] Start Model Finalization...
[VAI INFO] Model Finalization Done.
[VAI INFO] Quantization Finished.


2024-07-30 22:10:21.214590: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


176/176 [==============================] - 22s 89ms/step - loss: 0.0219 - accuracy: 0.9943

***************** Summary *****************
Test Quantized model accuracy: 0.9943
Quantized model saved to /workspace/grocery_store_trained/quant_model/quantized_resnet50_grocery.h5
[VAI INFO] Update include_bias_corr: False
[VAI INFO] Update include_fast_ft: False
[VAI INFO] Update include_cle: False
[VAI INFO] Standalone activation `softmax` layer dense_1_softmax is not supported.
[VAI INFO] Inspect Results:
[MODEL INFO]:
________________________________________________________________________________________________________________________
Model Name: model
________________________________________________________________________________________________________________________
ID          Name                    Type                    Device      Notes                                           
0/127       input_wrapper_for_input_InputLayer              INPUT                                  